In [3]:
import random
import numpy as np
import math
import pandas as pd
from sklearn.metrics import mean_squared_error
import numdifftools as nd
import inspect
from matplotlib import pyplot as plt
import statistics

In [4]:
random.seed(10)

# (A)

Assumptions:
- X distributed uniformly from (0,1) INCLUSIVE

## getData(N, var):

In [5]:
def getData(N, var=1.0):
    
    if(N<1 or var<0):
        return "ERROR"
    
    
    xes = []
    yes = []
    zes = []
    
    for i in range(N):
        x = random.uniform(0, 1)
        z = np.random.normal(loc=0.0, scale=math.sqrt(var))
        y = math.cos(2*math.pi*x)+z
        
        xes.append(x)
        yes.append(y)
        #zes.append(z)
        
    #df = pd.DataFrame(list(zip(xes,yes,zes)), columns =['X', 'Y','Z'])
    df = pd.DataFrame(list(zip(xes,yes)), columns =['X', 'Y'])
    
    return df
        

In [6]:
# CHECK

test_df = getData(10,2.0)

test_df

,X,Y
0,0.571403,-1.982518
1,0.428889,-1.816969
2,0.578091,0.825640
3,0.206098,-1.421772
4,0.813321,0.251154
5,0.823589,-2.326334
6,0.653473,-3.295630
7,0.160230,0.939546
8,0.520669,-1.051852
9,0.327773,1.506792


# (B)

## Helper Functions

In [7]:
def getPolynomialY(poly_coefs, x):
    
    y = 0
    
    for i in range(len(poly_coefs)):
        y += poly_coefs[i] * (x**i)
        
    return y

In [8]:
def getPrediction(coefs, xes):
    
    y_preds = []
    
    for x in xes:
        y_preds.append(getPolynomialY(coefs,x))
        
    return y_preds

In [9]:
# CHECK

# y = 2 + 3x + (-5)x^2 + 0x^3 + 6x^4
# Online Calculator: (2,84) , (-0.1, 1.6506)

print("x = 2, y =(true) 84 =(pred) ", getPolynomialY([2, 3, -5, 0, 6], 2))
print("x = (-0.1), y =(true) 1.6506 =(pred) ", getPolynomialY([2, 3, -5, 0, 6], -0.1))

x = 2, y =(true) 84 =(pred)  84
x = (-0.1), y =(true) 1.6506 =(pred)  1.6505999999999998


## getMSE(df, poly_coefs):

In [10]:
def getMSE(df, poly_coefs):
    
    N = len(df['X'])
    
    y_preds = getPrediction(poly_coefs, df['X'])
        
    y_residual = df['Y'] - y_preds
    
    return np.sum(np.dot(y_residual.T , y_residual)) / len(df['Y'] - y_residual)

In [11]:
# CHECK

Y_pred = []

for i in range(len(test_df['X'])):
    Y_pred.append( getPolynomialY([1,2,3], test_df['X'][i]) )

print(getMSE(test_df,[1,2,3]), "(pred)=(true)", mean_squared_error(test_df['Y'],Y_pred))

18.93404880271201 (pred)=(true) 18.93404880271201


# (C)

## Helper Functions

In [12]:
def transpose(matrix):
    return np.array(matrix).T.tolist()

In [13]:
def augmentedXDataframe(df, d):
    
    returnDf = df.copy()
    
    for i in range(d+1):
        returnDf['X_'+str(i)] = [x**i for x in df['X']]
        
    return returnDf

In [14]:
#Check 

augmented_test_df = augmentedXDataframe(test_df, 3)   
augmented_test_df

,X,Y,X_0,X_1,X_2,X_3
0,0.571403,-1.982518,1.0,0.571403,0.326501,0.186563
1,0.428889,-1.816969,1.0,0.428889,0.183946,0.078892
2,0.578091,0.825640,1.0,0.578091,0.334190,0.193192
3,0.206098,-1.421772,1.0,0.206098,0.042476,0.008754
4,0.813321,0.251154,1.0,0.813321,0.661491,0.538005
5,0.823589,-2.326334,1.0,0.823589,0.678299,0.558639
6,0.653473,-3.295630,1.0,0.653473,0.427026,0.279050
7,0.160230,0.939546,1.0,0.160230,0.025674,0.004114
8,0.520669,-1.051852,1.0,0.520669,0.271097,0.141152
9,0.327773,1.506792,1.0,0.327773,0.107435,0.035214


In [102]:
def MiniBatchStochasticGradientDescent(df, d, learning_rate, epochs, details=False):
    
    augmented_df = augmentedXDataframe(df, d)      #Add polynomial X terms to dataframe
    
    N = len(augmented_df['X'])
    
    # START FROM A RANDOM NON-ZERO THETA _____________________
    
    theta = []
    
    for i in range(d+1):
        num = 0
        while num==0:
            num = random.randint(-10,10)
        theta.append(num)
        
    if details: print("Random Theta: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
    
    # UPDATE THETA EPOCHS TIMES ______________________________
    
    for k in range(epochs):
        
        if details: print("_______________________________________________________________")
            
        # Choose random indexes in N for batch
        randomIndexes = []
        
        if N in [2]:                  # choose appropriate batch size
            batch_size= 1
        elif N in [5,10]:
            batch_size=2
        elif N in [20,50]:
            batch_size=5
        elif N in [100,200]:
            batch_size=10
        
        for i in range(batch_size):  
            randomIndexes.append(random.randint(0, N-1))
        
        for j in range(d+1):  # for each coefficient
            
            derivative = 0
            
            # GET DERIVATIVE for theta[j]
            for i in randomIndexes:
                derivative += 2 * (df['Y'][i] - getPolynomialY(theta, df['X'][i])) * augmented_df['X_'+str(j)][i]
            
            # UPDATE THETA
            theta[j] += learning_rate * derivative
        
        if details: print("Epoch: "+str(k)+", Coefs: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
        
    return theta
    
    
    return estim_poly_coefs

#MiniBatchStochasticGradientDescent(test_df, 3, 0.01, 1500, True)

## fitData(df, d, var, gd_type, learning_rate, mini_batch_size, epochs, testing_size, details, weightDecay):

In [103]:
def fitData(df, d, var, weightDecay, learning_rate=0.01, mini_batch_size = 10, epochs = 1500, testing_size=2000, details=False):

    # (1)
    if weightDecay:
        estim_poly_coefs = MiniBatchStochasticGradientDescent_WeightDecay(df, d, learning_rate, epochs, details)
    else:
        estim_poly_coefs = MiniBatchStochasticGradientDescent(df, d, learning_rate, epochs, details)
    
    # (2)
    E_in = getMSE(df, estim_poly_coefs)
    
    #(3)
    E_out = getMSE(getData(testing_size, var), estim_poly_coefs)
        
    return (estim_poly_coefs, E_in, E_out)

In [104]:
# Check

fitData(df=test_df, d=2, weightDecay=False, learning_rate=0.01, testing_size=2000, epochs = 1500, var=2.0, details=True)

Random Theta:  [10, -7, -10] MSE: 29.91258
_______________________________________________________________
Epoch: 0, Coefs:  [9.860099939020031, -6.994899294857371, -9.97234342577885] MSE: 28.79595
_______________________________________________________________
Epoch: 1, Coefs:  [9.682244829604253, -7.094817390271859, -10.030728313276764] MSE: 27.05715
_______________________________________________________________
Epoch: 2, Coefs:  [9.594767643336526, -7.068277922042244, -9.994234676823778] MSE: 26.49035
_______________________________________________________________
Epoch: 3, Coefs:  [9.420745125479531, -7.147982483211167, -10.03616042429579] MSE: 24.99009
_______________________________________________________________
Epoch: 4, Coefs:  [9.259301834108399, -7.232616252297846, -10.08192183810036] MSE: 23.66624
_______________________________________________________________
Epoch: 5, Coefs:  [9.06066321101388, -7.321035459992952, -10.122908363513455] MSE: 22.2044
______________________

Epoch: 181, Coefs:  [4.857780299733474, -5.090667876344504, -7.056906572609438] MSE: 7.41337
_______________________________________________________________
Epoch: 182, Coefs:  [4.695301260293708, -5.136319966474965, -7.072027471991853] MSE: 7.14362
_______________________________________________________________
Epoch: 183, Coefs:  [4.688367738426853, -5.084033319159193, -7.0181814731348595] MSE: 7.10237
_______________________________________________________________
Epoch: 184, Coefs:  [4.813829340485912, -4.985703198785242, -6.940058479328518] MSE: 7.29993
_______________________________________________________________
Epoch: 185, Coefs:  [4.7483052755276765, -5.0232951334424145, -6.962478472622017] MSE: 7.17999
_______________________________________________________________
Epoch: 186, Coefs:  [4.625580857660553, -5.054965665068815, -6.9727579753156865] MSE: 6.98129
_______________________________________________________________
Epoch: 187, Coefs:  [4.618379817840618, -5.00348302373

_______________________________________________________________
Epoch: 328, Coefs:  [2.927616491834218, -3.748848484804346, -5.252809277269037] MSE: 4.15469
_______________________________________________________________
Epoch: 329, Coefs:  [2.9100114150481136, -3.750250045802854, -5.252604524780007] MSE: 4.14651
_______________________________________________________________
Epoch: 330, Coefs:  [3.024108109310678, -3.6692795612682967, -5.192624764580118] MSE: 4.17226
_______________________________________________________________
Epoch: 331, Coefs:  [3.1301932546667492, -3.59375079684291, -5.136520953986733] MSE: 4.25188
_______________________________________________________________
Epoch: 332, Coefs:  [3.218772062828006, -3.524404927750488, -5.081489435854854] MSE: 4.35927
_______________________________________________________________
Epoch: 333, Coefs:  [3.1371172551285493, -3.575629734879362, -5.114391669085841] MSE: 4.25467
_______________________________________________________

Epoch: 474, Coefs:  [1.9334464018477595, -2.7986712813528207, -4.065840658894426] MSE: 2.99399
_______________________________________________________________
Epoch: 475, Coefs:  [1.8998818549475753, -2.802973458831589, -4.0659569804646205] MSE: 2.98955
_______________________________________________________________
Epoch: 476, Coefs:  [1.985543061630705, -2.744604892394124, -4.021944635286162] MSE: 2.98205
_______________________________________________________________
Epoch: 477, Coefs:  [1.9808697666132833, -2.7513472809343207, -4.027137066984396] MSE: 2.98314
_______________________________________________________________
Epoch: 478, Coefs:  [1.9632016038074624, -2.741460211470411, -4.0174028172585015] MSE: 2.97164
_______________________________________________________________
Epoch: 479, Coefs:  [1.9593259975464072, -2.7478626047499994, -4.022434464989908] MSE: 2.97324
_______________________________________________________________
Epoch: 480, Coefs:  [1.9169908029502525, -2.7710

Epoch: 577, Coefs:  [1.6508918136402948, -2.092208841178664, -3.2308010716499225] MSE: 2.6444
_______________________________________________________________
Epoch: 578, Coefs:  [1.6495881411574247, -2.062830035941379, -3.200963222126006] MSE: 2.64902
_______________________________________________________________
Epoch: 579, Coefs:  [1.6529161260751166, -2.053090669983715, -3.188534749658294] MSE: 2.65441
_______________________________________________________________
Epoch: 580, Coefs:  [1.5795850709669934, -2.0968812617151853, -3.215478711742497] MSE: 2.58576
_______________________________________________________________
Epoch: 581, Coefs:  [1.515188385226438, -2.126957478926505, -3.230253698519685] MSE: 2.54283
_______________________________________________________________
Epoch: 582, Coefs:  [1.5091120426985063, -2.1278921442082717, -3.2304033057276462] MSE: 2.53942
_______________________________________________________________
Epoch: 583, Coefs:  [1.4946527860533763, -2.134173

...

Epoch: 986, Coefs:  [0.6830688890925285, -1.2787263915290936, -2.0448371255582547] MSE: 2.16049
_______________________________________________________________
Epoch: 987, Coefs:  [0.7009347807039147, -1.2789266808346054, -2.049695334375624] MSE: 2.16919
_______________________________________________________________
Epoch: 988, Coefs:  [0.658463246129359, -1.2973107895447662, -2.0579662924656783] MSE: 2.14577
_______________________________________________________________
Epoch: 989, Coefs:  [0.6660057493934728, -1.2929596918060098, -2.055389582113151] MSE: 2.1497
_______________________________________________________________
Epoch: 990, Coefs:  [0.6519026133949151, -1.3051989467466087, -2.0630579675910683] MSE: 2.14194
_______________________________________________________________
Epoch: 991, Coefs:  [0.6323055267718907, -1.3163268863849158, -2.068302313672854] MSE: 2.13349
_______________________________________________________________
Epoch: 992, Coefs:  [0.5791828733803408, -1.3

Epoch: 1152, Coefs:  [0.6265043264112038, -1.0135551046394518, -1.6922955341483146] MSE: 2.25426
_______________________________________________________________
Epoch: 1153, Coefs:  [0.5717502142907284, -1.0356940699933366, -1.7059182956251988] MSE: 2.19938
_______________________________________________________________
Epoch: 1154, Coefs:  [0.6339101217492544, -1.0007132515060255, -1.682310963667177] MSE: 2.26879
_______________________________________________________________
Epoch: 1155, Coefs:  [0.5684437050019474, -1.024112860625687, -1.6930115013742313] MSE: 2.20356
_______________________________________________________________
Epoch: 1156, Coefs:  [0.5640487905869221, -1.0216868136952335, -1.689535646011476] MSE: 2.20188
_______________________________________________________________
Epoch: 1157, Coefs:  [0.5743272339500376, -1.0204663839694292, -1.6925412635763217] MSE: 2.20937
_______________________________________________________________
Epoch: 1158, Coefs:  [0.5057716273270

Epoch: 1327, Coefs:  [0.12013178474042356, -1.0954958699006236, -1.5715426140354267] MSE: 2.0715
_______________________________________________________________
Epoch: 1328, Coefs:  [0.09163331683868045, -1.1020595936185331, -1.5733574195284863] MSE: 2.07783
_______________________________________________________________
Epoch: 1329, Coefs:  [0.11095668676219075, -1.0912086559824703, -1.5670943799877455] MSE: 2.0723
_______________________________________________________________
Epoch: 1330, Coefs:  [0.12596081915661023, -1.0888872437601558, -1.5673680370080687] MSE: 2.06979
_______________________________________________________________
Epoch: 1331, Coefs:  [0.07707388331458861, -1.103633664376618, -1.5725575517762842] MSE: 2.08144
_______________________________________________________________
Epoch: 1332, Coefs:  [0.05020718357294564, -1.1203935133009784, -1.5835498570718187] MSE: 2.09328
_______________________________________________________________
Epoch: 1333, Coefs:  [0.0717437

Epoch: 1468, Coefs:  [0.053557709807727935, -1.0422050952669542, -1.4423219652272683] MSE: 2.06902
_______________________________________________________________
Epoch: 1469, Coefs:  [0.13067587776934836, -0.9903637845542811, -1.4057560339132773] MSE: 2.06442
_______________________________________________________________
Epoch: 1470, Coefs:  [0.18786595865893949, -0.958072892923024, -1.3818241069730393] MSE: 2.07733
_______________________________________________________________
Epoch: 1471, Coefs:  [0.1798843741145961, -0.9664564547888835, -1.3862225414797076] MSE: 2.07414
_______________________________________________________________
Epoch: 1472, Coefs:  [0.1884982379556314, -0.943309063578117, -1.3642648962523058] MSE: 2.0811
_______________________________________________________________
Epoch: 1473, Coefs:  [0.19698049617421584, -0.9433871948876988, -1.3654685604639374] MSE: 2.08324
_______________________________________________________________
Epoch: 1474, Coefs:  [0.14265372

([-0.05079589511922756, -1.1667819751875135, -1.521352966350741],
 2.137790857889051,
 4.419507755297133)

# (D)

## experiment(N, d, var, M, details, weightDecay):

In [105]:
def experiment(N, d, var, weightDecay, M=50, details=False):
    
    training_data_df = getData(N,var)
    
    polynomials = []
    E_ins = []
    E_outs = []
    
    if details: print("_____________________________________________________________________ START TRIALS\n")
    
    for i in range(M):
        if details: print("______________________________________ M="+str(i+1))
        
        (p,ein,eout) = fitData(training_data_df, d, var, weightDecay, learning_rate=0.01, epochs=1000, testing_size=2000, details=False)
        
        polynomials.append(p)
        E_ins.append(ein)
        E_outs.append(eout)
        
        if details: print("FINAL MODEL: Coefs: ", p, " E_in: "+ str(ein)+" E_out: "+str(eout))
        
    if details: print("\n_____________________________________________________________________ AVERAGE POLYNOMIALS\n")
    
    E_in_bar = statistics.mean(E_ins)
    E_out_bar = statistics.mean(E_outs)
    
    average_polynomial = []
    transpose_coefs = transpose(polynomials)
    
    for i in range(d+1):
        average_polynomial.append(statistics.mean(transpose_coefs[i]))
        
    if details: print("AVERAGE MODEL: Coefs: ", average_polynomial, " E_in_bar: "+ str(round(E_in_bar,5))+" E_out_bar: "+str(round(E_out_bar,5)))
        
        
    if details: print("\n_____________________________________________________________________ TESTING\n")
        
    testing_data_df = getData(2000, var)
    
    E_bias = getMSE(testing_data_df, average_polynomial)
    
    if details: print("E_bias: "+str(E_bias))
    
    return (E_in_bar, E_out_bar, E_bias)

In [99]:
#Check

experiment(N=200, d=2, var=0.01, M=5, weightDecay=False, details=True)

_____________________________________________________________________ START TRIALS

______________________________________ M=1
FINAL MODEL: Coefs:  [1.2248357485481354, -7.738893040235268, 7.8002385998611885]  E_in: 0.05664656320765484 E_out: 0.059646358682262646
______________________________________ M=2
FINAL MODEL: Coefs:  [0.876019031867781, -5.581453919385868, 5.655645844988197]  E_in: 0.11277989502785114 E_out: 0.11466229894024914
______________________________________ M=3
FINAL MODEL: Coefs:  [-0.03919415216127226, -0.41994159640080714, 0.8059350052688187]  E_in: 0.4471903625025413 E_out: 0.4406688658596603
______________________________________ M=4
FINAL MODEL: Coefs:  [0.6164972801399378, -4.067887750833734, 3.952708670159416]  E_in: 0.19522518773292158 E_out: 0.20661393956088056
______________________________________ M=5
FINAL MODEL: Coefs:  [0.17798236044130017, -1.9969611230003836, 2.0309621980682064]  E_in: 0.3292986263397603 E_out: 0.35749153058628724

___________________

(0.22822812696214584, 0.23581659872586797, 0.19475808205745238)

# (E) Mini-Batch Stochastic Gradient Descent

In [100]:
Nes = [2,5,10,20,50,100,200]
Des = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
Vares = [0.01, 0.1, 1]

In [ ]:
nn = []
dd = []
varvar = []
einbars = []
eoutbars = []
ebiass = []

for myN in Nes:
    for myD in Des:
        for myVar in Vares:
            print("_________________________________________________________________________")
            print("N = "+str(myN)+"; d = "+str(myD)+"; var = "+str(myVar))
            [E_in_bar, E_out_bar, E_bias] = experiment(N=myN, d=myD, var=myVar, M=50, weightDecay=False, details=False)
            print("E_in_bar = "+str(E_in_bar)+"; E_out_bar = "+str(E_out_bar)+"; E_bias = "+str(E_bias))
            
            nn.append(myN)
            dd.append(myD)
            varvar.append(myVar)
            einbars.append(E_in_bar)
            eoutbars.append(E_out_bar)
            ebiass.append(E_bias)
            
output_df = pd.DataFrame(list(zip(nn,dd,varvar,einbars,eoutbars,ebiass)), columns =['N','d','var','E_in_bar','E_out_bar','E_bias']) 

_________________________________________________________________________
N = 2; d = 0; var = 0.01
E_in_bar = 0.009012420637927671; E_out_bar = 0.7173065614656363; E_bias = 0.7174693485726947
_________________________________________________________________________
N = 2; d = 0; var = 0.1
E_in_bar = 0.4727127147333778; E_out_bar = 0.6146728943724057; E_bias = 0.6142390651872667
_________________________________________________________________________
N = 2; d = 0; var = 1
E_in_bar = 0.09104310357328523; E_out_bar = 2.681490396719548; E_bias = 2.6757954414484955
_________________________________________________________________________
N = 2; d = 1; var = 0.01
E_in_bar = 0.017239800662287273; E_out_bar = 5.051278300690487; E_bias = 0.6781493309422969
_________________________________________________________________________
N = 2; d = 1; var = 0.1
E_in_bar = 0.621714092405919; E_out_bar = 2.4923149678778973; E_bias = 1.145041769032065
______________________________________________________

E_in_bar = 0.016503342117446973; E_out_bar = 18.217256570138648; E_bias = 2.7526638928427465
_________________________________________________________________________
N = 2; d = 14; var = 1
E_in_bar = 0.12922630744557506; E_out_bar = 25.828980977629286; E_bias = 1.2347400153717774
_________________________________________________________________________
N = 2; d = 15; var = 0.01
E_in_bar = 3.1362376346614205e-07; E_out_bar = 6.1603921109010145; E_bias = 0.37689175459852375
_________________________________________________________________________
N = 2; d = 15; var = 0.1
E_in_bar = 0.5849920159704031; E_out_bar = 36.51484710961919; E_bias = 8.596302146245726
_________________________________________________________________________
N = 2; d = 15; var = 1
E_in_bar = 1.5358768850584439; E_out_bar = 41.72425739917626; E_bias = 8.037098679506048
_________________________________________________________________________
N = 2; d = 16; var = 0.01
E_in_bar = 0.1218548231300213; E_out_bar = 28.85

E_in_bar = 0.5212856059687248; E_out_bar = 1.044293108663861; E_bias = 0.3544166807404124
_________________________________________________________________________
N = 5; d = 8; var = 0.1
E_in_bar = 0.18363861151921534; E_out_bar = 6.602153725123958; E_bias = 0.7060759637471047
_________________________________________________________________________
N = 5; d = 8; var = 1
E_in_bar = 0.42641392182468113; E_out_bar = 23.5971160649896; E_bias = 5.508685221624345
_________________________________________________________________________
N = 5; d = 9; var = 0.01
E_in_bar = 0.31652632898713434; E_out_bar = 2.015365305765047; E_bias = 0.654956862433
_________________________________________________________________________
N = 5; d = 9; var = 0.1
E_in_bar = 0.3776325086711698; E_out_bar = 3.8960089029927873; E_bias = 0.6785919465170507
_________________________________________________________________________
N = 5; d = 9; var = 1
E_in_bar = 0.3401200826486332; E_out_bar = 2.5984779319318885; E_

In [ ]:
output_df.to_excel('output.xlsx') 

# (F) Mini-Batch Stochastic Gradient Descent with Weight Decay

In [ ]:
def MiniBatchStochasticGradientDescent_WeightDecay(df, d, learning_rate, epochs, weight_decay=0.1, details=False):
    
    augmented_df = augmentedXDataframe(df, d)      #Add polynomial X terms to dataframe
    
    N = len(augmented_df['X'])
    
    # START FROM A RANDOM NON-ZERO THETA _____________________
    
    theta = []
    
    for i in range(d+1):
        num = 0
        while num==0:
            num = random.randint(-10,10)
        theta.append(num)
        
    if details: print("Random Theta: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
    
    # UPDATE THETA EPOCHS TIMES ______________________________
    
    for k in range(epochs):
        
        if details: print("_______________________________________________________________")
            
        # Choose random indexes in N for batch
        randomIndexes = []
        
        if N in [2]:                  # choose appropriate batch size
            batch_size= 1
        elif N in [5,10]:
            batch_size=2
        elif N in [20,50]:
            batch_size=5
        elif N in [100,200]:
            batch_size=10
        
        for j in range(d+1):  # for each coefficient
            
            derivative = 0
            
            # GET DERIVATIVE for theta[j]
            for i in randomIndexes:
                derivative = (weight_decay * derivative) + 2 * (df['Y'][i] - getPolynomialY(theta, df['X'][i])) * augmented_df['X_'+str(j)][i]
            
            # UPDATE THETA
            theta[j] += learning_rate * derivative
        
        if details: print("Epoch: "+str(k)+", Coefs: ", theta, "MSE: "+ str(round(getMSE(augmented_df, theta), 5)))
        
    return theta
    
    
    return estim_poly_coefs

#MiniBatchStochasticGradientDescent_WeightDecay(test_df, 3, 0.01, epochs=1500, details=True)

In [ ]:
nn = []
dd = []
varvar = []
einbars = []
eoutbars = []
ebiass = []

for myN in Nes:
    for myD in Des:
        for myVar in Vares:
            print("_________________________________________________________________________")
            print("N = "+str(myN)+"; d = "+str(myD)+"; var = "+str(myVar))
            [E_in_bar, E_out_bar, E_bias] = experiment(N=myN, d=myD, var=myVar, M=50, weightDecay=True, details=False)
            print("E_in_bar = "+str(E_in_bar)+"; E_out_bar = "+str(E_out_bar)+"; E_bias = "+str(E_bias))
            
            nn.append(myN)
            dd.append(myD)
            varvar.append(myVar)
            einbars.append(E_in_bar)
            eoutbars.append(E_out_bar)
            ebiass.append(E_bias)
            
output_df = pd.DataFrame(list(zip(nn,dd,varvar,einbars,eoutbars,ebiass)), columns =['N','d','var','E_in_bar','E_out_bar','E_bias']) 

In [ ]:
output_df.to_excel('output_weight_decay.xlsx') 